In [1]:
#'C:\\Users\\ac36345\\Desktop\\nlp code\\glove.6B\\glove.6B.50d.txt',encoding="utf8"
import numpy as np
import json 
import os
import theano
import theano.tensor as T
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.util import shuffle
from word2vec import get_wikipedia_data,find_analogies

class Glove:
    def __init__(self,D,V,contect_sz):
        self.D = D 
        self.V = V
        self.context_sz = context_sz
    def fit(self,sentences,cc_matrix=None,learning_rate=10e-5,reg=0.1,xmax=100,alpha=0.75,epochs=10,gd=False,use_theano=True):
        t0 = datetime.now()
        V=self.V
        D=self.Dn
        if not os.path.exists(cc_matrix):
            x = np.zeros((V,V))
            N = len(sentences)
            print("number of sentences of process:",N)
            it =0
            for sentence in sentences:
                it +=1
                if it % 10000 ==0:
                    print("processed",it,"/",N)
                n = len(sentence)
                for i in xrange(n):
                    wi = sentence[i]
                # i,j is not the word ind
                    start = max(0,i-self.context_sz)
                    end =   min(n,i+self.context_sz)
                    if i-self.context_sz < 0 :
                        points = 1.0 / (i+1)
                        X[wi,0] +=points
                        X[0,wi] +=points
                    if i+self.context_sz > n:
                        points = 1.0 / (n-i)
                        X[wi,1] +=points
                        X[1,wi] +=points
                    #left side
                    for j in xrange(start , i):
                        wj = sentence[j]
                        points = 1.0 /(i-j)
                        X[wi,wj] +=points
                        X[wj,wi] +=points
                   #right side
                    for j in xrange(i+1,end):
                        wj = sentence[j]
                        points = 1.0 /(j-i)
                        X[wi,wj] +=points
                        X[wj,wi] +=points
            np.save(cc_matrix,X)
        else:
            X = np.load(cc_matrix)
            print("max value in X:",X.max())
            fx = np.zeros((V,V))
            fx[X,xmax] = (X[X < xmax] / float(xmax)) ** alpha
            fx[X >= xmax] = 1
            logX = np.log(X+1)
            print("time to build co-occurrence matrix:",(datetime.now() - t0)
            w = np.random.randn(V,D)/np.sqrt(V+D)
            b = np.zeros(V)
            U = np.random.randn(V,D)/np.sqrt(V+D)
            c = np.zeros(V)
            mu = logX.mean()
            if gd and use_theano:
                  pass
          costs = []
          sentence_indexes = range(len(sentences))
          for epoch in xrange(epochs):
              delta = W.dot(U.T) +b.reshape(V,1) + c.reshape(1,V) + mu - logX
              cost = (fX * delta * delta).sum()
              costs.append(cost)
              print("epoch:",epoch,"cost:",cost)
              if gd:
               # print('******')
                if use_theano:
                    pass
                else: 
                    oldW =  W.copy()
                    for i in xrange(V):
                        W[i] -= learning_rate * (fX[i,:]*delta[i,:]).dot(U)
                        W -= learning_rate*reg*W
                    for i in xrange(V):
                        b[i] -= learning_rate * fX[i,:].dot(delta[i,:])
                        b -= learning_rate*reg*b
                    for j in xrange(V):
                        U[i] -= learning_rate * (fX[:,j]*delta[:,j]).dot(oldW)
                        U -= learning_rate*reg*U
                    for j in xrange(V):
                        c[j] -= learning_rate *fX[:,j].dot(delta[:,j])
                        c -= learning_rate*reg*c
               # print('&&&&&')
              else:
                pass
          self.w = w
          self.U = U
          plt.plot(costs)
          plt.show()
      def save(self,fn):
          arrays = [self.W,self.U.T]
          np.savex(fn,*arrays)

    def main(we_file,w2i_file,n_files=50):
        cc_matrix = "cc_matrix_%s.npy" %n_files
        if os.path.exists(cc_matrix):
            with open(w2i_file) as f:
                word2idx =  json.load(f)
            sentences =[]
        else:
            sentences,word2idx = get_wikipedia_data(n_files=n_files,n_vocab=2000)
            with open(w2i_file,'w') as f:
                json.dump(word2idx,f)
        
        V = len(word2idx)
        model = Glove(80,V,10)
        model.fit(
             sentences=sentences,
             cc_matrix = cc_matrix,
             learning_rate = 3*10e-5,
             reg = 0.01,
             epochs = 2000,
             gd = True,
             use_theano = False,
        )
        model.save(we_file)
    if __name__ == '__main__':
        we ='glove_model_50.npz'
        w2i = 'glove_model_50.json'
        main(we,w2i)
        for concat in (True,False):
            print("** concat:",concat)
            find_analogies('king','man','woman',concat,we,w2i)

                      


SyntaxError: invalid syntax (<ipython-input-1-423359fa1aa3>, line 66)

In [1]:
from __future__ import print_function,division
from builtins import range
from nltk.corpus import brown
import operator

KEEP_WORDS = set([
'king','man','queen','woman',
'italy','rome','france','paris',
'london','britain','england',])

def get_sentences():
    return brown.sents()

def get_sentences_with_word2idx():
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START':0,'END': 1}
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                word2idx[token] = i
                i +=1
            indexed_sentence.append(word2idx[token])
        indexed_sentences.append(indexed_sentence)
    print('vocab size:',i)
    return indexed_sentences,word2idx
def get_sentences_with_word2idx_limit_vocab(n_vocab=2000,keep_words=KEEP_WORDS):
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START':0,'END': 1}
    idx2word = ['START','END']
    word_idx_count = {
      0: float('inf'),
      1: float('inf'),
    }
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                idx2word.append(token)
                word2idx[token] = i
                i += 1
            idx = word2idx[token]
            word_idx_count[idx] = word_idx_count.get(idx,0) + 1
            indexed_sentence.append(idx)
        indexed_sentences.append(indexed_sentence)
    for word in keep_words:
        word_idx_count[word2idx[word]] = float('inf')
        # sort technique operator.itemgetter(1)
    sorted_word_idx_count = sorted(word_idx_count.items(), key = operator.itemgetter(1),reverse = True)
    word2idx_small = {}
    new_idx = 0
    idx_new_idx_map = {}

    for idx,count in sorted_word_idx_count[:n_vocab]:
        word = idx2word[idx]
        print(word, count)
        word2idx_small[word] = new_idx
        idx_new_idx_map[idx] = new_idx
        new_idx +=1
    word2idx_small['UNKNOWN'] = new_idx
    unknown = new_idx
    assert('START' in word2idx_small)
    assert('END' in word2idx_small)
    for word in keep_words:
        assert(word in word2idx_small)
    sentences_small = []
    for sentence in indexed_sentences:
        if len(sentence) > 1:
            new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
            sentences_small.append(new_sentence)
    return sentences_small, word2idx_small